# Get data and packages

In [1]:
%%capture
%%bash
# only a single file
pip install -U pyarrow numpy koalas
# aws s3 cp  s3://xdss-public-datasets/demos/taxi_parquet/data_0.parquet datasets/taxi_parquet/data_0.parquet

In [14]:
!cp ../datasets/taxi_parquet/data_0.parquet datasets/taxi_parquet/data_0.parquet

In [20]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os
import os


instance_type = 'c5d2xlarge' # change this
results_bucket = f"s3://vaex-sagemaker-demo/benchmarks" # change this

name = 'spark-koalas'
data_path = 'datasets/taxi_parquet/data_0.parquet'
output_file = f'{name}_{instance_type}_1m_results.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {}
single_repetition = 1
statistic_repetition = 5

long_min = -74.05
long_max = -73.75
lat_min = 40.58

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks, name):
    results = pd.DataFrame.from_dict(benchmarks, orient='index')
    results.columns = [name]
    return results

def persist():
    gc.collect()
    get_results(benchmarks, name).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, repetitions=1, **kwargs):
    times = []
    for i in range(repetitions):
        start_time = time.time()
        ret = f(df, **kwargs)
        times.append(time.time()-start_time)
    benchmarks[name] = np.mean(times)
    persist()
    print(f"{name} took: {benchmarks[name]}")
    return benchmarks[name]


!mkdir -p results
!mkdir -p datasets
print(f"{name} - test for {single_repetition} repetitions for join and groupby and {statistic_repetition} repetitions for statistics")

spark-koalas - test for 1 repetitions for join and groupby and 5 repetitions for statistics


# Benchmark

In [21]:
import pandas as pd
import numpy as np
import databricks.koalas as ks
from pyspark.sql import SparkSession

# Load data
data = ks.read_parquet(data_path)
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1000000 with 18 columns


In [22]:
def read_file_parquet(df=None):
    return ks.read_parquet(data_path)

benchmark(read_file_parquet, df=data, name='read_file', repetitions=statistic_repetition)

read_file took: 0.21591391563415527


0.21591391563415527

In [23]:
def count(df=None):
    return len(df)

benchmark(count, df=data, name='count', repetitions=statistic_repetition)

count took: 0.08967857360839844


0.08967857360839844

In [24]:
def mean(df):
    return df.fare_amount.mean()

benchmark(mean, df=data, name='mean', repetitions=statistic_repetition)

mean took: 0.227276611328125


0.227276611328125

In [25]:
def standard_deviation(df):
    return df.fare_amount.std()

benchmark(standard_deviation, df=data, name='standard deviation', repetitions=statistic_repetition)

standard deviation took: 0.15072994232177733


0.15072994232177733

In [26]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean()

benchmark(mean_of_sum, df=data, name='sum columns mean', repetitions=statistic_repetition)


sum columns mean took: 0.15644373893737792


0.15644373893737792

In [27]:
def sum_columns(df):
    return df.fare_amount + df.trip_distance

benchmark(sum_columns, df=data, name='sum columns', repetitions=statistic_repetition)

sum columns took: 0.005517196655273437


0.005517196655273437

In [28]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean()

benchmark(mean_of_product, df=data, name='product columns mean', repetitions=statistic_repetition)

product columns mean took: 0.12584896087646485


0.12584896087646485

In [29]:
def product_columns(df):
    return df.fare_amount * df.trip_distance

benchmark(product_columns, df=data, name='product columns', repetitions=statistic_repetition)

product columns took: 0.0012202739715576171


0.0012202739715576171

In [39]:
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    return np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation', repetitions=single_repetition)

arithmetic operation took: 0.12942957878112793


0.12942957878112793

In [40]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = np.multiply(np.arctan2(np.sqrt(temp), np.sqrt(1-temp)),2) 
    return ret.mean()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean', repetitions=single_repetition)

arithmetic operation mean took: 0.8055226802825928


0.8055226802825928

In [41]:
def value_counts(df):
    return df.fare_amount.value_counts()

benchmark(value_counts, df=data, name='value counts', repetitions=statistic_repetition)

value counts took: 0.05088129043579102


0.05088129043579102

In [42]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              })

benchmark(groupby_statistics, df=data, name='groupby statistics', repetitions=single_repetition)

groupby statistics took: 0.2589566707611084


0.2589566707611084

In [61]:
other = groupby_statistics(data)
other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

In [65]:
def join_data(df, other):
    return df.join(other, on='passenger_count')

benchmark(join_data, data, name='join', repetitions=single_repetition, other=other)

join took: 0.15024518966674805


0.15024518966674805

In [66]:
def join_count(df, other):
    return len(df.join(other, on='passenger_count'))

benchmark(join_count, data, name='join count', repetitions=single_repetition, other=other)

join count took: 1.0953681468963623


1.0953681468963623

## Filtered data

In [67]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 0 MB


In [68]:
expr_filter = (data.pickup_longitude > long_min)  & (data.pickup_longitude < long_max) & \
                  (data.pickup_latitude > lat_min)    & (data.pickup_latitude < lat_max) & \
                  (data.dropoff_longitude > long_min) & (data.dropoff_longitude < long_max) & \
                  (data.dropoff_latitude > lat_min)   & (data.dropoff_latitude < lat_max)

In [69]:
def filter_data(df):
    return df[expr_filter]

benchmark(filter_data, data, name='filter data', repetitions=statistic_repetition)

filter data took: 0.33727312088012695


0.33727312088012695

In [70]:
filterd = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

cleaned 145 mb


In [ ]:
benchmark(mean, filterd, name='filterd mean', repetitions=statistic_repetition)
benchmark(standard_deviation, filterd, name='filtered standard deviation', repetitions=statistic_repetition)
benchmark(mean_of_sum, filterd, name ='filtered sum columns mean', repetitions=statistic_repetition)
benchmark(sum_columns, df=filterd, name='filtered sum columns', repetitions=statistic_repetition)
benchmark(mean_of_product, filterd, name ='filterd product columns mean', repetitions=statistic_repetition)
benchmark(product_columns, df=filterd, name='filterd product columns', repetitions=statistic_repetition)
benchmark(mean_of_complicated_arithmetic_operation, filterd, name='filterd arithmetic operation mean', repetitions=single_repetition)
benchmark(complicated_arithmetic_operation, filterd, name='filterd arithmetic operation', repetitions=single_repetition)
benchmark(value_counts, filterd, name ='filtered value counts', repetitions=statistic_repetition)
benchmark(groupby_statistics, filterd, name='filtered groupby statistics', repetitions=single_repetition)
other = groupby_statistics(filterd)
benchmark(join_data, filterd, name='filtered join', repetitions=single_repetition, other=other)
benchmark(join_count, filterd, name='filtered join count', repetitions=single_repetition, other=other)
print('Done!')
print('spark-koalas')
benchmarks

filterd mean took: 6.406383943557739
filtered standard deviation took: 5.537730360031128
